In [1]:
from math import pi

In [2]:
from fasta import *
import alignment
import utilities
from checkGenome import *
from ipywidgets import widgets
from ipywidgets import *
from traitlets import *
from IPython.display import display

# Loading the files

The first thing we want to do is load in our files. Loading them in this way actually loads them into a dictionary where the keys are the record IDs and the values are the full records.

In [4]:
# Load the 2U1 files
# full_record = SeqIO.to_dict(SeqIO.parse("files/2U1_all_candidates_PSI_BLAST_unique.fasta", "fasta"))
# full_record = SeqIO.to_dict(SeqIO.parse("files/Python_bivittaus_full_PSI.fasta", "fasta"))
# full_record = SeqIO.to_dict(SeqIO.parse("files/2U1_and_2U1_like_candidates_BLAST_results_unique_X_seqs_removed.fasta", "fasta"))
# full_record = SeqIO.to_dict(SeqIO.parse("files/fGMC-3-2_GOX_GDH-3_SS_only_species.fasta", "fasta"))
# full_record = SeqIO.to_dict(SeqIO.parse("files/71AV.fasta", "fasta"))
#full_record = SeqIO.to_dict(SeqIO.parse("files/homo_sapiens.fasta", "fasta"))

# full_record = SeqIO.to_dict(SeqIO.parse("files/candidates/regextest.fasta", "fasta"))

FileNotFoundError: [Errno 2] No such file or directory: 'files/2U1_all_candidates_PSI_BLAST_unique.fasta'

#### How many sequences do we have in the full records?

In [ ]:
print (len(full_record))

In [ ]:
for record in full_record:
    print (full_record[record].description)
    print (full_record[record].name)

# Creating subsets of the records
### Including / excluding based on header annotations

Now the workflow moves to including and excluding certain sequences. `subset_records` allows us to provide a list of terms which we either want or don't want in the header description. We can also give a minimum length for sequences to meet for inclusion.

We don't ever alter the original `full_record`, we just create new dictionary objects that are subsets.

We can either provide arguments directly to the function or we can pass in a list variable, such as `header_terms`.

In the example below `only_2U1_records` is set to only include sequences which have either '2U1' or '2U1-like' in the header. And `filtered_records` will contain the full set of sequences as we are not providing a length minimum (and the default is 0) and we are passing in the currently empty `header_terms`.

In [ ]:
# A blank list to hold terms we want to exclude or include
header_terms = []

In [ ]:
only_2U1_records = subset_records("2U1", "2U1-like", records=full_record, length=400, mode="include")
filtered_records = subset_records(*header_terms, records=full_record, length=500, mode='exclude')

print ("The number of sequences with either 2U1 or 2U1-like in the header is %s " % (len(only_2U1_records)))
print ("The number of sequences we've filtered is %s which should be equal to %s" % (len(filtered_records), len(full_record)))

### Adding terms to the `header_terms` variable
The following section makes it easy to add in terms to the `header_terms` variable and to save these files for later use.

Let's first print out the terms in our variable and the length of it. As you add to the list you can always come back and rerun this cell to peek inside the `header_terms` variable

In [ ]:
print (header_terms)
print (len(header_terms))

The first thing we might be interested in doing is to print out the header information of the sequences we currently have.

In [ ]:
for record in filtered_records:
    print (filtered_records[record].description)

The cell below will add items to our `header_terms` variable. Hit run on the cell and you'll see an input box - simply add words seperated by a space that you want to add.

In [ ]:
add = widgets.Text()
display(add)

def handle_submit(sender):
    for item in add.value.split():
        header_terms.append(item)
    print (header_terms)
add.on_submit(handle_submit)

And then we can also remove 

In [ ]:
remove = widgets.Text()
display(remove)

def handle_submit(sender):
    for item in remove.value.split():
        header_terms.remove(item)
    print (header_terms)

remove.on_submit(handle_submit)

Below is that cell that lets us check all the words in `header_terms` so far.

In [ ]:
print (header_terms)
print (len(header_terms))

Have a play around with adding and removing words to the `header_terms` list and then the following cells illustrate how it can be used.

Make the `header_terms` list contain just the terms "2B4" and "2B4-like" and then we'll create a new record called `only_2B4_records`

In [ ]:
only_2B4_records = subset_records(*header_terms, records=full_record, mode='include')
for record in only_2B4_records:
    print (only_2B4_records[record].description)

### Saving and loading the header terms variable

In [ ]:
utilities.saveHeaderTerms(header_terms, "files/headerterms.txt")

In [ ]:
header_terms = utilities.loadHeaderTerms("files/headerterms.txt")

### Subsetting record files using regular expressions
Typing all of the particular items we want to include or exclude can be time-consuming, and often we want to include or exclude all of the members of a family. So we can use regular expressions in `subset_records_with_regex` and only supply the first part of the family name and have it automatically match to all headers that contain text starting with that first part.

For example - excluding "2J" would exclude "2J6", "2J2", and "2J2-like" (as well as others)

In [ ]:
test_records = subset_records_with_regex("subfamily A", "subfamily a","subfamily 2 J19", "Cyp2c38", "subfamily D", "subfamily d", "subfamily E", "subfamily e", "subfamily AA", "subfamily C", "subfamily X", "subfamily x", "subfamily c", "subfamily J", "subfamily j", "subfamily AD", "subfamily ad", "subfamily B", "subfamily b", "subfamily k", "subfamily K", "2U", "2D", "2J", "2B", "2A", "2B", "2C", "2D", "2E", "2F", "2G", "2H", "2I", "2J", "2K", "2L", "2M", "2N", "2O", "2P", "2Q", "2R", "2S", "2T", "2V", "2W", "2X", "2Y", "2Z", "1A", "1B", "76C", "84A", "98A", "304a1", "305a1", "306a1", "CYP17A", "303a1", "307a1", "83B", "81E", "81d1", "81D1", "18",    records=full_record, mode="exclude")
# test_records = subset_records_with_regex("2U", records=full_record, mode="include")
# test_records = subset_records("uncharacterized", "25-hydroxylase", "partial", "hypothetical", "unnamed", "Cyp2r1", records=test_records, mode='exclude')

print (len(test_records)) 

In [ ]:
species_counts = build_species_count(records=test_records)


In [ ]:
for item in test_records:
    print (test_records[item].description)

# Evaluating how many hits per species

`build_species_count` builds a dictionary which has the set of unique species as its keys and a list of the sequence IDs that belong to each unique species as its . So we can use it to easily see how many unique species we have and which species are over represented.

In [ ]:
species_counts = build_species_count(records=full_record)
print("There are %s unique species in our dataset." % (len(species_counts)))

### Plotting the frequency of proteins per species
`plot_record_number` is a function that plots the numbers of IDs per species. We can set a minimum number of IDs that a species must have in order to be plotted.

In [ ]:
plotthis = plot_record_number(species_counts, "Bar", min=3)
py.iplot(plotthis, filename='inline_bar')

In [ ]:
plotthis = plot_record_number(species_counts, "Bar", min=2)
py.iplot(plotthis, filename='inline_bar')

In [ ]:
plotthis = plot_record_number(species_counts, "Bar", min=5)
py.iplot(plotthis, filename='inline_bar')

#### We can also just extract the names using `get_species_name`, which also accepts a minimum number of IDs required and can print out the number of counts per each species

In [ ]:
species_names = get_species_names(species_counts, min=1)
for name in species_names:
    print (name)

In [ ]:
species_names_with_counts = get_species_names(species_counts, min=3, counts=True)
for name in species_names_with_counts:
    print (name)

### Counting the total number of sequences with multiple hits
`count_ids` is a function that counts the total number of sequences in a species count dictionary, not just the number of unique species.

As before, it can also take a minimum number of IDs required

In [ ]:
min_num = 5
print ("There are %s total sequences in our filtered dataset." % (count_ids(species_counts)))
print ("There are %s total sequences in our filtered dataset that have %d or more IDs per species." % (count_ids(species_counts, min=min_num), min_num))

# Generating datasets containing information about species with multiple hits
For each species that has more than the given number of hits, we create 
1. A FASTA file of the protein sequences from that species
2. An alignment of the protein sequences
3. An information file telling use where in the genome the protein maps to
4. A visual diagram of the genome mapping the proteins to the genome

In [ ]:
def generate_multiple_hit_data(species_names, species_counts, full_record, file_path):
    id_dict = {}
#     for name in species_names:
#         seqs = map_species_to_records(species_counts[name], full_record)
#         write_fasta(seqs, file_path + name + " sequences")
#         alignmentFile = alignment.alignWithMAFFT(file_path + name + " sequences")
#         alignment.writeAlignment(alignmentFile, file_path + name + ".aln", "fasta")
        

    check_genomic_location(species_counts, min=1, file_path=file_path +" gene locations ")
    check_genomic_location(species_counts, min=1, visualise="linear")


species_names = get_species_names(species_counts, min=1)
generate_multiple_hit_data(species_names, species_counts, full_record, "files/multiple_hits/")

Or we could just use parts of this function. The cell below will just print out the locations of the proteins in the genome. We could save this to disk by providing an argument to the `file_path` variable or visualise it by providing either 'linear' or 'circular' to the `visualise` variable.

In [ ]:
check_genomic_location(species_counts, min=5)

# Saving the records to FASTA files
Because `filtered_records` just contains the species name and IDs of these species, we need to map these IDs back to their full records. We can use the function `map_ids_to_records` which allows for us to select all the records in `filtered_ids` or just the unique species.

In [ ]:
filtered_records = map_ids_to_records(species_counts, full_record)
filtered_records_unique = map_ids_to_records(species_counts, full_record, unique=True)

# Check that the numbers are correct
print (len(filtered_records))
print (len(filtered_records_unique))

And now we can save these records to a new FASTA file using `write_fasta`

In [ ]:
write_fasta(filtered_records, "files/2U1_BLAST_smaller_records.fasta")
# write_fasta(filtered_records_unique, "files/2U1_BLAST_filtered_records_unique.fasta")

We can also use the function `map_species_to_records` to just map a particular species to a FASTA file.

In [ ]:
priapulus_caudatus = map_species_to_records(species_counts['Priapulus caudatus'], full_record)
write_fasta(priapulus_caudatus, "files/priapulus_caudatus.fasta")